In [ ]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 获取订单数据

In [ ]:
sql = """
select om.id order_id, tpmn.name 手机机型,tod.sku_attributes,
om.status, om.user_id,om.user_mobile,
case om.status when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 ,
tol.go_express_date 寄出物流时间, tol.go_express_number 寄出快递单号, om.merchant_name,
tmu.true_name, om.order_number, tmu.id_card_num, tmu.mobile,
tmuic.id_card_address 户籍地址, tol.delivery_detail_address 收货地址,
tmuc.contact_mobile 紧急联系人,
tps.sku_rent_type 套餐类型, om.all_money 签约租金, tod.new_actual_money 签约买断价, tprm.all_rental 总租金,
om.buy_service_product, tso.price 碎屏险, om.create_time, tos.sort, tos.refund_date, 
tos.reality_refund_date,
tos.real_pay_money 当月实收, tos.money 当月应收, om.has_actual, om.update_time 更新日期, top.total_describes,
pa.name as activity_name        -- 活动名称

from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_product_sku tps on tps.id=tod.sku_id
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
left join db_digua_business.t_product tp on tp.id=tod.product_id
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_member_user_id_card tmuic on tmuic.id_card_no=tmu.id_card_num
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
left join db_digua_business.t_order_stages tos on tos.order_id=om.id
left join db_digua_business.t_order_verify_queue tovq on tovq.order_id=om.id
left join db_digua_business.t_service_order tso on tso.order_id=om.id
left join db_digua_business.t_merchant_cash_water tmcw on tmcw.order_id=om.id
left join db_digua_business.t_order_relet tor on tor.order_id=om.id
left join db_digua_business.t_member_user_contacts tmuc on tmuc.user_id=om.user_id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
where om.status in (2, 3, 4, 5, 6, 8, 15)
-- and om.merchant_name in ("小蚂蚁租机", "人人享租", "喜卓灵租机", "喜卓灵新租机")
and date_format(om.create_time, '%Y-%m-%d') >= '2024-09-01'
and date_format(om.create_time, '%Y-%m-%d') <= '2025-08-01'

"""

df_order = clean.query(sql)
df_order.shape

In [ ]:
df_order.columns


In [ ]:
# 仅筛选订单号在df_tar中存在的数据

df_order = df_order[df_order['order_number'].isin(list_tar)]
df_order.shape

# 数据处理

In [ ]:
df = df_order.copy()

# 查看重复的(order_id, sort)组合
duplicates = df[df.duplicated(subset=['order_id', 'sort'], keep=False)]
if not duplicates.empty:
    print("发现重复数据，数量：", len(duplicates))
    # 可以选择保留最新的记录（根据实际业务逻辑调整）
    # 这里假设reality_refund_date不为空的是更新的记录
    df = df.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
    # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
    df = df.drop_duplicates(subset=['order_id', 'sort'], keep='first')

df.loc[:, 'now_day'] = pd.to_datetime(datetime.now().date())
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df.loc[:,"颜色"]=df.apply(lambda x:clean.getcolor(x["sku_attributes"]),axis=1)
df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:, '机型组合'] = df.手机机型+'_'+df.内存


dict_rent_rule = {
    "R00001": "租完需归还",
    "R00002": "租完即送",
    "R00003": "随租随还",
    "R00004": "随时续租",
    "R00005": "随时买断",
    "R00006": "租完归还/随时续租",
    "RO0007": "租完归还/可买断",
    "R00008": "租完归还/随时续租/随时买断",
    "R00009": "租完即送/随租随还",
    "R00010": "租完即送/随时买断",
    "R00011": "租完即送/随租随还/随时买断",
    "R00012": "随租随还/随时续租",
    "R00013": "随租随还/随时买断",
    "R00014": "随租随还/随时续租/随时买断",  # 修正笔误：原“随时随租”→“随时续租”
    "R00015": "随时续租/随时买断"
}
df["套餐"]=df["套餐类型"].map(dict_rent_rule)
df.loc[:, 'refund_date'] = df.refund_date.dt.date
df.loc[:, 'reality_refund_date'] = df.reality_refund_date.dt.date
df.loc[:, '更新日期'] = df.更新日期.dt.date
df["更新日期"]=pd.to_datetime(df["更新日期"],errors="coerce")


# 首逾：sort=2，refund_date<now_day|reality_refund_date>refund_date
df.loc[:, 'overdue_day'] = np.where(df.reality_refund_date.isna(), np.where(df.refund_date<df.now_day, df.now_day-df.refund_date, 0), 
                                    np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0))
df.loc[:, 'overdue_day'] = df.overdue_day.apply(lambda x: x.days)
# 先取出逾期天数大于0的记录，首次逾期日期即为最小的refund_date

# 对于有逾期的记录，计算该订单的首次逾期日期（即逾期天数>0的最小应还日期）
df_with_overdue = df[df['overdue_day'] > 0]
first_overdue_date = df_with_overdue.groupby('order_id')['refund_date'].min()

df.loc[:, '首次逾期日期']= df['order_id'].map(first_overdue_date)
df.loc[:, '首次逾期日期'] = pd.to_datetime(df['首次逾期日期'], errors='coerce')
df.loc[:,'买断'] = np.where(df.has_actual==1, "买断", "非买断")
df['买断日期'] = df['更新日期'].where(df['has_actual'] == 1, pd.NaT)
# 强制确认类型（可选，确保万无一失）
df['买断日期'] = pd.to_datetime(df['买断日期'], errors='coerce')

# 如果订单id所表示的行存在首次逾期日期不为空的记录，则按refund_date选取第一个首次逾期日期不为空的记录，否则取第一条记录即可
# 优先保留首次逾期记录
df_overdue = df[df['overdue_day'] > 0].copy()
# 每个订单按应还日期排序，取最早的一条逾期记录
df_first_overdue = df_overdue.sort_values(['order_id', 'refund_date']).drop_duplicates(subset=['order_id'], keep='first')

# 对于没有逾期记录的订单，保留应还日期最早的记录
no_overdue_order_ids = set(df['order_id']) - set(df_first_overdue['order_id'])
df_no_overdue = df[df['order_id'].isin(no_overdue_order_ids)]
df_no_overdue = df_no_overdue.sort_values(['order_id', 'refund_date']).drop_duplicates(subset=['order_id'], keep='first')


# 合并结果
df_ans = pd.concat([df_first_overdue, df_no_overdue], ignore_index=True)
# df = clean.order_drop_duplicates(df)
# df_overdue.shape
df_ans.shape

In [ ]:
# df_ans.head()

In [ ]:
df_tem = df_ans[['order_id', 'status2', '手机机型', '内存', '颜色', '机型组合', '寄出物流时间', '寄出快递单号', 'merchant_name', 'true_name', 'order_number',
            'id_card_num', 'mobile', '户籍地址', '收货地址', '紧急联系人', '套餐', '签约租金', '签约买断价', '总租金', '碎屏险', '下单日期', '首次逾期日期',
            'overdue_day', '当月应收', '当月实收', '买断', '买断日期']].rename(columns={'status2': '订单状态', '寄出物流时间': '发货日期', '寄出快递单号': '顺丰单号',
            'merchant_name': '店铺名称', 'true_name': '姓名', 'order_number': '订单号', 'id_card_num': '身份证号', 'mobile': '电话号码', 'overdue_day': '逾期天数'
            })

## 数据清洗(df_sort)

In [ ]:
sql_sort = """
select
tpr.order_id, tos.sort, tos.refund_date, tos.money, tos.reality_refund_date, tos.real_pay_money,
tpr.pay_type,om.order_number
from db_digua_business.t_pay_record tpr
left join db_digua_business.t_order_stages tos on tos.order_id=tpr.order_id
left join db_digua_business.t_order om on om.id=tpr.order_id
where om.status in (2, 3, 4, 5, 6, 8, 15)
-- and om.merchant_name in ("小蚂蚁租机", "人人享租", "喜卓灵租机", "喜卓灵新租机")
and date_format(om.create_time, '%Y-%m-%d') >= '2024-09-01'
and date_format(om.create_time, '%Y-%m-%d') <= '2025-08-01'
and tos.status = 3 -- 已支付
"""
df_sort = clean.query(sql_sort)
df_sort.shape

In [ ]:
# # 读入excel文件,41条记录
# df_tar = pd.read_excel('F:\需求\廖志彧需求\迪瓜.xlsx')
# list_tar = df_tar.order_number.to_list()#.shape
# df_sort_t = df_sort[df_sort['order_number'].isin(list_tar)]
# df_sort_t.shape

In [ ]:

# # 1. 首先检查并处理重复数据
# # 查看重复的(order_id, sort)组合
# duplicates = df_sort_t[df_sort_t.duplicated(subset=['order_id', 'sort'], keep=False)]
# if not duplicates.empty:
#     print("发现重复数据，数量：", len(duplicates))
#     # 可以选择保留最新的记录（根据实际业务逻辑调整）
#     # 这里假设reality_refund_date不为空的是更新的记录
#     df_sort_t = df_sort_t.sort_values(['order_id', 'sort', 'refund_date'], ascending=[True, True, False])
#     # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
#     df_sort_t = df_sort_t.drop_duplicates(subset=['order_id', 'sort'], keep='first')
#     print(df_sort_t.shape)

# payment_mapping = {
#     "WX": "微信",
#     "WXXCX": "微信小程序",
#     "ZFB": "支付宝",
#     "ZFBYSQ": "支付宝预授权",
#     "ZFBXCX": "支付宝小程序支付",
#     "DYXCX": "抖音小程序支付",
#     "DY_HX_PAY": "抖音小程序核销支付",
#     "YE": "余额支付",
#     "WXZFF": "微信支付分",
#     "ZFBZQQ": "支付宝周期扣款",
#     "XX": "线下支付",
#     "UNIONXX": "银联线下",
#     "DEPOSIT": "押金抵扣",
#     "RY_PAY": "第三方代扣",
#     "ANT_PAY": "蚂蚁链代扣",
#     "ZYX_PAY": "智赢行支付",
#     "ZFB_WAP_PAY": "支付宝h5支付",
#     "ZFBYWD": "支付宝业务单代扣",
#     "ZFXXSK": "线下(对资方)支付",
#     "XXX": "线下(对维客)支付",
#     "XX_MERCHANT": "线下(对商家)支付",
#     "UNICOM_PAY": "联通号卡支付",
#     "STAGE_PAY": "租期抵扣",
#     "OVERDUE_POOL_PAY": "逾期扣款池抵扣",
#     "BROKERAGE_REFUND": "佣金退还",
#     "JD_AGREEMENT_PAY": "京东签约支付",
#     "JD_PAY": "京东主动支付",
#     "ZFB_APP_PAY": "支付宝APP SDK主动支付",
#     "WX_APP_PAY": "微信APP SDK主动支付"
# }

# df_sort_t

In [ ]:
# df_renamed = df_sort_t.rename(columns={
#     'sort': '期数',
#     'pay_type': '支付方式'
# })

# df_renamed["支付方式"] = df_renamed["支付方式"].map(payment_mapping)
# # 4. 选择需要的列
# df_selected = df_renamed[['order_number', '期数','支付方式']]


In [ ]:

# # 5. 使用pivot_table处理（更稳健，可处理潜在的重复值）
# df_pivot = df_selected.pivot_table(
#     index='order_number',
#     columns='期数',
#     values=['期数', '支付方式'],
#     aggfunc='first'  # 对重复值取第一个
# )
# print(df_pivot)
# # 6. 调整列顺序
# columns = []
# for period in range(1, 13):
#     columns.extend([
#         ('期数', period),
#         ('支付方式', period)
#     ])

# # 确保所有期数都存在（即使某些期数没有数据）
# missing_periods = set(range(1,13)) - set(df_pivot.columns.get_level_values(1))
# for period in missing_periods:
#     for col in ['期数', '支付方式']:
#         df_pivot[(col, period)] = np.nan

# df_pivot = df_pivot.reindex(columns=columns)

# # 遍历1-12期，填充期数列
# for period in range(1, 13):
#     # 给('期数', period)这一列赋值为period
#     df_pivot[('期数', period)] = period

# # 7. 展平列名
# flattened_columns = []
# for period in range(1, 13):
#     flattened_columns.extend(['期数', '支付方式'])

# df_pivot.columns = flattened_columns

# # 8. 最终结果以订单号为索引
# result_df = df_pivot.reset_index().set_index('order_number')
# result_df.head()

In [ ]:
current_date = pd.Timestamp.now()
def get_order_status(row):
    # 处理空值（默认实际还款金额为0，应还款日为极早日期）
    money = row['money'] if pd.notnull(row['money']) else 0
    real_received = row['real_pay_money'] if pd.notnull(row['real_pay_money']) else 0
    refund_date = row['refund_date'] if pd.notnull(row['refund_date']) else None # 默认为极早日期（视为逾期）
    reality_refund_date = row['reality_refund_date'] if pd.notnull(row['reality_refund_date']) else None
    
    
    # 1. 判断是否逾期, reality_refund_date为空，判断refund_date是都小于now_day，小于则逾期，大于则未逾期，reality_refund_date不为空，判断reality_refund_date是否大于refund_date，大于则逾期，小于则未逾期
    # 1. 判断是否逾期
    if reality_refund_date is not None:
        # 如果已有实际还款日期，比较实际还款日期与应还日期
        is_overdue = reality_refund_date > refund_date
    elif refund_date is not None:
        # 如果没有实际还款日期，比较当前日期与应还日期
        is_overdue = current_date > refund_date
    else:
        # 如果应还日期也为空，默认为未逾期
        is_overdue = False
    
    # 2. 判断支付状态
    if real_received == 0:
        pay_status = "待支付"
    elif 0 < real_received < money:
        pay_status = "部分支付"
    elif real_received == money:
        pay_status = "已支付"
    else:
        # 特殊情况（如超额支付，按实际业务定义，这里暂归为"部分支付"）
        pay_status = "部分支付"
    
    # 3. 组合逾期状态和支付状态
    if not is_overdue:  # 未逾期
        if pay_status == "已支付":
            return "未逾期已支付"
        elif pay_status == "待支付":
            return "未逾期待支付"
        else:  # 部分支付
            return "未逾期部分支付"
    else:  # 逾期
        if pay_status == "已支付":
            return "逾期已支付"
        elif pay_status == "待支付":
            return "逾期待支付"
        else:  # 部分支付
            return "逾期部分支付"

# 应用函数生成订单状态列未逾期已支付、未逾期待支付、逾期已支付、逾期待支付、未逾期部分支付、逾期部分支付
df_sort['订单状态'] = df_sort.apply(get_order_status, axis=1)

In [ ]:
# 如果real_pay_money列值不为0，则优惠价为money-real_pay_money，否则为0
df_sort['优惠价'] = np.where(df_sort['real_pay_money'] != 0, df_sort['money'] - df_sort['real_pay_money'], 0)

# 3. 重命名列名
df_renamed = df_sort.rename(columns={
    'sort': '期数',
    'refund_date': '应还款日',
    'money': '应还款金额',
    'reality_refund_date': '实际还款日',
    'real_pay_money': '实际还款金额',
    'pay_type': '支付方式'
})

df_renamed["支付方式"] = df_renamed["支付方式"].map(payment_mapping)
# 4. 选择需要的列
df_selected = df_renamed[['order_id', '期数', '应还款日', '应还款金额', '实际还款日', '实际还款金额', '支付方式', '优惠价', '订单状态']]

# 5. 使用pivot_table处理（更稳健，可处理潜在的重复值）
df_pivot = df_selected.pivot_table(
    index='order_id',
    columns='期数',
    values=['期数', '应还款日', '应还款金额', '实际还款日', '实际还款金额', '支付方式', '优惠价', '订单状态'],
    aggfunc='first'  # 对重复值取第一个
)
print(df_pivot)
# 6. 调整列顺序
columns = []
for period in range(1, 13):
    columns.extend([
        ('期数', period),
        ('应还款日', period),
        ('应还款金额', period),
        ('实际还款日', period),
        ('实际还款金额', period),
        ('支付方式', period),
        ('优惠价', period),
        ('订单状态', period),
    ])

# 确保所有期数都存在（即使某些期数没有数据）
missing_periods = set(range(1,13)) - set(df_pivot.columns.get_level_values(1))
for period in missing_periods:
    for col in ['期数', '应还款日', '应还款金额', '实际还款日', '实际还款金额', '支付方式', '优惠价', '订单状态']:
        df_pivot[(col, period)] = np.nan

df_pivot = df_pivot.reindex(columns=columns)

# 遍历1-12期，填充期数列
for period in range(1, 13):
    # 给('期数', period)这一列赋值为period
    df_pivot[('期数', period)] = period

# 7. 展平列名
flattened_columns = []
for period in range(1, 13):
    flattened_columns.extend(['期数', '应还款日', '应还款金额', '实际还款日', '实际还款金额', '支付方式', '优惠价', '订单状态'])

df_pivot.columns = flattened_columns

# 8. 最终结果以订单号为索引
result_df = df_pivot.reset_index().set_index('order_id')
result_df.head()

In [ ]:
result_df.shape

In [ ]:
# 按照订单号将数据合并
df_res = pd.merge(df_tem, result_df, how='left', on=['order_id'])


In [ ]:
df_res.shape

In [ ]:
# df_res.to_excel(f'F:\需求\倩姐需求\客户成交信息表.xlsx', index=False)